In [25]:
import h5py
import numpy as np

from pathlib import Path


In [29]:
data_dir = Path("./data/")
assert data_dir.is_dir()
intra_dir = data_dir / "Intra"
cross_dir = data_dir / "Cross"
intra_train_glob = list((intra_dir / "train").glob("*.h5"))
intra_test_glob = list((intra_dir / "test").glob("*.h5"))
len(intra_train_glob)

32

In [40]:
def load_labels(path: Path) -> np.ndarray:
    *task, subject_identifier, chunk = path.stem.split("_")
    if "rest" in task:
        y = 0
    elif 'math' in task:
        y = 1
    elif 'working' in task:
        y = 2
    elif 'motor' in task:
        y = 3
    else:
        assert False, 'unknown task'
    return np.array([y, int(subject_identifier), int(chunk)])

In [43]:
def load_h5(path: Path) -> np.ndarray:
    with h5py.File(path) as f:
        keys = f.keys()
        assert len(keys) == 1, f"Only one key per file, right? {intra_train_glob[0]}"
        matrix = f.get(next(iter(keys)))[()]
    return matrix


intra_train_X = np.stack(list(map(load_h5, intra_train_glob)))
intra_train_labels = np.stack(list(map(load_labels, intra_train_glob)))
intra_train_X.shape, intra_train_labels.shape

((32, 248, 35624), (32, 3))

In [42]:
intra_test_X = np.stack(list(map(load_h5, intra_test_glob)))
intra_test_labels = np.stack(list(map(load_labels, intra_test_glob)))
intra_test_X.shape, intra_test_labels.shape

((8, 248, 35624), (8, 3))